In [66]:
import datetime 

future_static_fields = ['FUT_DLV_DT_LAST', 'FUT_DLV_DT_FIRST', 'FUT_CNVS_FACTOR', 'FUT_CTD_ISIN',
                            'FUT_FIRST_TRADE_DT', 'LAST_TRADEABLE_DT', 'BAD_FLD']
     
bond_fut_hist_fields = ['PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_LAST', "PX_VOLUME", 'OPEN_INT', "FUT_CTD_CUSIP", "FUT_CNVS_FACTOR", 
                        "FUT_IMPLIED_REPO_RT", "FUT_ACTUAL_REPO_RT", "CONVENTIONAL_CTD_FORWARD_FRSK"]

pricing_fields = ['MODEL PRICE', 'ADJUSTED MODEL PRICE', 'MODEL PRICE ADJUSTMENT', 'MONETARY VALUE', 'PV',
                      'CTD YIELD TO MATURITY', 'DV01', 'DV01 CONTRACT', 'DV01 100', 'INVOICED AMOUNT', 'CTD BOND',
                      'CTD CONVERSION FACTOR', 'ASSUMED DELIVERY DATE', 'LAST TRADING DATE', 'NOTIONAL', 'UPDATED',
                      'ID']
 
bulk_field = "FUT_CHAIN"
ticker = "TYZ3 Comdty"  
today_dt = datetime.datetime.today()
start_dt = datetime.datetime(2013, 1, 1)

#### tia.bbg

In [ ]:
from tia.bbg import LocalTerminal

In [75]:
# Historical
historyresponse = LocalTerminal.get_historical(ticker, bond_fut_hist_fields[0], start=start_dt, end=today_dt,
                                                       ignore_security_error=True,
                                                       ignore_field_error=True)
history = historyresponse.as_frame()
history.columns = [bond_fut_hist_fields[0]]
history

,PX_OPEN
date,
2023-05-09,116.796875
2023-05-30,115.062500
2023-06-02,114.562500
2023-06-05,114.312500
2023-06-07,114.015625
...,...
2023-11-09,108.437500
2023-11-10,107.562500
2023-11-13,107.343750


#### blp Module

In [50]:
from blp import blp
import re

blpparser = blp.BlpParser(raise_security_errors=False)
bquery = blp.BlpQuery(parser=blpparser).start()

def get_ref(ticker, fields):
    while len(fields) > 0:
        try:
            data = bquery.bdp(ticker, fields)
            break
        except TypeError as e:
            if 'fieldException' in str(e):
                match = re.search("'fieldId': '([^']+)'", str(e))
                err_fld = match.group(1)
                fields.remove(err_fld)
        except Exception as e:
            if 'Session Not Started' in str(e):
                print('Session Not Started')
            else:
                print(e)
            break
    return data.to_dict(orient='records')[0]

def get_history(ticker, fld, start_dt, end_dt):
    while True:
        try:
            data = bquery.bdh(ticker, fld, start_dt.strftime("%Y%m%d"), end_dt.strftime("%Y%m%d"))
            return data
        except Exception as e:
            return None
    

In [63]:
get_ref(ticker, future_static_fields)

{'security': 'TYZ3 Comdty',
 'FUT_DLV_DT_LAST': Timestamp('2023-12-29 00:00:00'),
 'FUT_DLV_DT_FIRST': Timestamp('2023-12-01 00:00:00'),
 'FUT_CNVS_FACTOR': 0.9003,
 'FUT_CTD_ISIN': 'US91282CHW47',
 'FUT_FIRST_TRADE_DT': Timestamp('2023-03-23 00:00:00'),
 'LAST_TRADEABLE_DT': Timestamp('2023-12-19 00:00:00')}

In [96]:
today_str = today_dt.strftime("%Y%m%d")
start_str = start_dt.strftime("%Y%m%d")
df = bquery.bdh([ticker], [bond_fut_hist_fields[0]], start_str, today_str,)
df.set_index('date', inplace=True)
df.drop('security', axis=1, inplace=True)
df

In [98]:
bquery.bds(ticker, bulk_field)

,Security Description
0,TYZ3 Comdty
1,TYH4 Comdty
2,TYM4 Comdty
